In [ ]:
from models import UNet, PointSetGen

In [ ]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()

model = UNet().double()
criterion = nn.MSELoss()

# Move model and loss function to GPU
if use_gpu: 
    criterion = criterion.cuda()
    model = model.cuda()

# optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.1, weight_decay=0.01)

# https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-a-general-checkpoint-for-inference-and-or-resuming-training

checkpoint = torch.load('../checkpoints/model-epoch-7-losses-282.914.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
inc = checkpoint['epoch'] + 1 # increment depending on how many epochs we already completed